<a href="https://colab.research.google.com/github/AdiA32/ImageAesthetic/blob/main/Aesthetic_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [5]:
# Installs

!pip install --upgrade google-cloud-vision
!pip install mysql-connector-python-rf
!pip install -qU transformers torch datasets gdcm pydicom pinecone-client

In [ ]:
# Google Imports

import os
from google.cloud import vision
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine, text

# Embedding Imports
import torch
import skimage
import requests
import numpy as np
import pandas as pd
from PIL import Image
from io import BytesIO
import pinecone
import IPython.display as display
import matplotlib.pyplot as plt
from datasets import load_dataset
from collections import OrderedDict
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
import torchvision.transforms as T

import json
import os
import pinecone
import openai
import requests
import numpy as np
import itertools
import torch
import random
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from urllib.parse import urlparse
from pathlib import Path
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
from openai import OpenAI
from PIL import Image
from imageio import imread
from io import BytesIO
from google.colab import userdata


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


RuntimeError: ignored

In [ ]:
# Initialize Pinecone
pinecone.init(api_key=userdata.get('PINECONE_API_KEY'), environment="gcp-starter")

# Create a Pinecone index
index_name = 'aesthetic'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=512)

# Connect to your index
index = pinecone.Index(index_name)




#Need openaikey
openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model_id = "openai/clip-vit-base-patch32"

processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id)

# move model to device if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'

out = model.to(device)

# Create a Dataset of 10 Aesthetics
- Select 10 different fashion aesthetics (e.g., streetwear, vintage, boho, etc.).
- For each aesthetic, gather a minimum of 10 images (5 representing male fashion and 5 representing female fashion).
- Ensure these images are diverse and accurately represent the chosen aesthetic. singles?


In [7]:
styles = [
    "Classic Businesswear",
    "Streetwear",
    "Preppy",
    "Minimalist",
    "Skater",
    "Soft",
    "Athleisure",
    "Boho (Bohemian)",
    "Vintage",
    "Athletic"
]

#added gender so i can build better database
def get_styles(query, gender, num_results=5):
    url = "https://www.googleapis.com/customsearch/v1"

    full_query = f"{query} {gender} fashion"  # Include gender in the query

    params = {
        "q": full_query,
        "cx": userdata.get('GOOGLE_CSE_ID'),
        "key": userdata.get('GOOGLE_API_KEY'),
        "searchType": "image",
        "num": num_results
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        return [item["link"] for item in response.json()["items"]]
    else:
        print("Error: ", response.status_code)
        return []

#calling ^ and getting 5 women and 5 men items for each style
def get_style_images(styles, num_results_per_gender=5):
    images = []
    for style in styles:
        for gender in ["men", "women"]:
            try:
                temp = get_styles(style, gender, num_results_per_gender)
            except Exception as e:
                print(f"Error occurred for style: {style}, gender: {gender}, error: {e}")
                temp = []
            image = {
                "style": style,
                "gender": gender,
                "images": temp
            }
            images.append(image)
    return images


def download_images(image, base_folder, verbose=False):
    style = image['style']
    images = image['images']
    style_folder = Path(base_folder) / style

    # Create the folder for the style if it doesn't exist
    style_folder.mkdir(parents=True, exist_ok=True)
    output = []

    for url in images:
        try:
            # Parse the URL to get the image name
            image_name = Path(urlparse(url).path).name
            image_path = style_folder / image_name

            # Download the image
            response = requests.get(url)
            if response.status_code == 200:
                with open(image_path, 'wb') as file:
                    file.write(response.content)
                    output.append({"style": style, "image_name": image_name, "image_path": image_path, "url": url})
                if verbose:
                    print(f"Image saved at {image_path}")
            else:
                print(f"Failed to download {url}")

        except Exception as e:
            print(f"Error occurred: {e}")

    return output


In [ ]:
images = get_style_images(styles)

In [8]:

for image in images:
    print(image["style"])
    for img in image["images"]:
        print(img)
    print()

Classic Businesswear
https://m.media-amazon.com/images/I/61aT+97zQWL._AC_UY1000_.jpg
https://i.pinimg.com/564x/71/21/4a/71214a29ea3510acd739cd603f3c6f1a.jpg
https://m.media-amazon.com/images/I/81MWYrJ8BrL._AC_UY1000_.jpg
https://i.pinimg.com/736x/72/1b/4d/721b4d958ac1e7eaf304789f1577f16e.jpg
https://m.media-amazon.com/images/I/61i3bzLKIfL._AC_UF894,1000_QL80_.jpg

Classic Businesswear
https://d2w9m16hs9jc37.cloudfront.net/dimg/blog/2021/06/278a0084.jpg
https://i.pinimg.com/736x/cd/68/ea/cd68eab11e7b62bfcb67f57e2db912bf.jpg
https://i5.walmartimages.com/seo/YOTAMI-Dress-for-Women-2022-Autumn-and-Winter-Long-Sleeve-Pullver-Classic-Loose-Fashion-Casual-Deals-under-15-Blue-S-6XL_dbf1a423-6b0d-4ca0-b554-badd51dfd19c.a705425ca0bf2bbd9cdfe770811b21d0.jpeg?odnHeight=768&odnWidth=768&odnBg=FFFFFF
https://i.pinimg.com/736x/7a/f8/21/7af82197ab34029376a0ce88a68eac0e.jpg
https://i5.walmartimages.com/seo/YOTAMI-Dress-for-Women-2022-Autumn-and-Winter-Long-Sleeve-Pullver-Classic-Loose-Fashion-Casual-De

# Create Embeddings Using OpenAI GPT-4V (Turbo Preview Model) and Upsert to Pinecone
- Use the OpenAI GPT-4V (Turbo Preview model) to generate embeddings for each image. This model can create rich, descriptive embeddings that capture the essence of the images.
- Once the embeddings are generated, upsert (update or insert) them into your Pinecone index along with relevant metadata (like aesthetic type, gender representation, etc.).


Notes:
Embeddings, in the context of machine learning and particularly with your project involving OpenAI's GPT-4V and Pinecone, are a way of representing complex data, like images, in a form that a computer can understand and process efficiently. Here’s a more detailed explanation:

### What Are Embeddings?

1. **Numerical Representations**: Embeddings are numerical representations of data in a high-dimensional space. For images, this means converting the visual content into a list (or vector) of numbers.

2. **Capturing Essence**: These numerical vectors aim to capture the essence or key features of the data. For an image, this might include color schemes, shapes, textures, or even more abstract concepts like the style or mood of the image.

3. **Dimensionality Reduction**: Embeddings often involve reducing the dimensionality of the original data. An image, which is essentially a large array of pixel values, is transformed into a much smaller vector that still retains the important information.

### How Do They Work in Your Project?

In your project, you use OpenAI's GPT-4V to generate embeddings for images. Here's how that fits in:

1. **Image Processing**: When you pass an image to OpenAI's GPT-4V model, the model processes the image and extracts its key features.

2. **Vector Creation**: The model then represents these features as a vector of numbers. This vector is the "embedding" of the image.

3. **Uses of Embeddings**: These embeddings can be used for various purposes, such as:
   - **Similarity Comparison**: Comparing vectors to find similar images (images with similar vectors are likely to be visually or stylistically similar).
   - **Indexing and Retrieval**: In Pinecone, you store these embeddings so that you can later search through them efficiently. When you query the Pinecone index with an embedding, it retrieves items with similar embeddings.

### Why Use Embeddings?

1. **Efficiency**: Dealing with raw images directly is often computationally expensive and inefficient. Embeddings reduce the complexity of the data while retaining the essential information.

2. **Machine Learning Readiness**: Embeddings are a form of feature extraction, making the data ready for machine learning models, which typically work with numbers.

3. **Versatility**: The same embedding technique can be used for different types of data (like text, images, and audio), allowing for consistent processing and analysis methods.

### Conclusion

In summary, in your project, embeddings serve as a bridge between the rich, complex world of visual data and the numerical realm where machine learning algorithms operate. They allow you to efficiently store, search, and analyze images based on their content.

Retard terms:
In simpler terms, embeddings are like a special code that represents something complex in a way that computers can understand and work with easily.

Imagine you have a bunch of photos, and you want a computer to sort them based on their style. Photos are complicated — they have lots of colors, shapes, and details. A computer can't just look at a photo and understand it like we do. So, we turn each photo into an embedding, which is like a unique fingerprint for each photo.

Here's how it works:

1. **Turning Photos into Numbers**: A computer program looks at a photo and picks out its most important features, like colors, patterns, and shapes. Then, it turns these features into a list of numbers. This list of numbers is the embedding.

2. **Comparing Photos**: Now, each photo has its own list of numbers. The computer can easily compare these lists to see which photos are similar. Photos with similar lists of numbers are likely to have similar styles.

3. **Storing and Searching**: We can store these lists of numbers in a system. Later, if we want to find photos with a certain style, the computer can quickly search through these numbers and find what we're looking for.

In your project, you're using a special tool from OpenAI to create these number lists for each photo. Then, you use another tool called Pinecone to store and search through these lists. This makes it easier to organize and find photos based on their style!

In [ ]:
import openai
import base64

# Your OpenAI API key
openai.api_key = 'YOUR_OPENAI_API_KEY'

def get_image_embedding(image_path):
    """
    Generates an embedding for an image using OpenAI's API.

    :param image_path: Local path to the image.
    :return: Embedding of the image.
    """
    with open(image_path, 'rb') as image_file:
        image_base64 = base64.b64encode(image_file.read()).decode('utf-8')

    response = openai.Image.create_embedding(
        model="turbo-preview-model",  # Replace with the appropriate model name
        image_base64=image_base64
    )

    return response['data']


In [ ]:
def upsert_to_pinecone(id, embedding, metadata):
    """
    Upserts data to a Pinecone index.

    :param id: A unique identifier for the data.
    :param embedding: The embedding vector.
    :param metadata: Additional metadata to store with the embedding.
    """
    index.upsert(vectors={id: (embedding, metadata)})


4. Write a Query Function
Develop a function that takes an image as input. This function should use Visual Question Answering (VQA) techniques to analyze the image.
The function should classify the aesthetic of the outfit in the image based on its visual characteristics.
Utilize machine learning models or AI services capable of understanding fashion-related queries for this purpose.


cool idea: show pic of outfit i am wearing and get aesthetic?

In [12]:
import requests

def classify_outfit_aesthetic(image_path, api_key):
    """
    Classifies the aesthetic of an outfit in an image.

    :param image_path: Path to the image file.
    :param api_key: API key for the image analysis service.
    :return: Predicted aesthetic of the outfit.
    """
    # Load the image
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()

    # Prepare the request for the API
    # This part varies depending on the specific API you're using
    headers = {'Authorization': f'Bearer {api_key}'}
    payload = {'image': image_data}  # Adjust this according to the API's requirements

    # Send the request to the API
    response = requests.post('API_ENDPOINT', headers=headers, data=payload)

    # Process the response
    # The processing here will depend on the structure of the response from the API
    if response.status_code == 200:
        analysis_result = response.json()
        # Extract the relevant information from the analysis_result to determine the aesthetic
        # This might involve looking for specific keywords or features
        aesthetic = extract_aesthetic_from_result(analysis_result)
        return aesthetic
    else:
        print(f"Error in API call: {response.status_code}")
        return None

def extract_aesthetic_from_result(result):
    """
    Extracts the aesthetic from the analysis result.

    :param result: The result from the image analysis API.
    :return: The determined aesthetic.
    """
    # Implement logic to analyze the result and determine the aesthetic
    # This could be based on identified objects, colors, styles, etc.
    # For example:
    if 'vintage' in result['description']:
        return 'Vintage'
    elif 'sporty' in result['description']:
        return 'Athletic'
    # Add more conditions as needed
    else:
        return 'Unknown'

# Example usage
api_key = 'YOUR_API_KEY'
image_path = 'path/to/your/image.jpg'
aesthetic = classify_outfit_aesthetic(image_path, api_key)
print(f"The outfit's aesthetic is: {aesthetic}")


FileNotFoundError: ignored

5. Filter and Retrieve Correct Images from Pinecone Database
Once the aesthetic is determined, use metadata filtering to query your Pinecone index.
Retrieve images that match the classified aesthetic.
Ensure the results are diverse and representative of the aesthetic category.


SyntaxError: ignored